# Hands on tutorial for ImJoy

## Your first ImJoy plugin

In [30]:
from imjoy import api

class ImJoyPlugin():
    async def setup(self):
        pass

    async def run(self, ctx):
        pokemon = await api.prompt("What is your favorite Pokémon?", "Pikachu")
        await api.showMessage("Your have chose " + pokemon + " as your Pokémon.")

api.export(ImJoyPlugin())

<IPython.core.display.Javascript object>

<_GatheringFuture pending>

## Visualize an Image using Kaibu

Kaibu is an ImJoy plugin made for visualizing and anotating images.

Kaibu can be used independently (https://kaibu.org) or through its ImJoy applicaiton interface (API)

Full documentation including interactive examples can be found at: https://kaibu.org/docs/

The following examples shows how to load an image from the human protein atlas ("https://images.proteinatlas.org/61448/1319_C10_2_blue_red_green.jpg")


In [31]:
from imjoy import api

class ImJoyPlugin():
    async def setup(self):
        pass

    async def run(self, ctx):
        viewer = await api.createWindow(src="https://kaibu.org/#/app")
        await viewer.view_image("https://images.proteinatlas.org/61448/1319_C10_2_blue_red_green.jpg", name="example image")

api.export(ImJoyPlugin())

<IPython.core.display.Javascript object>

<_GatheringFuture pending>

## Visualize an image from your computer

Kaibu can also display images in numpy array in Python, therefore you can use the Python module called `imageio` (https://imageio.readthedocs.io/en/stable/) to load a wide range of image formats (including tiff) and display it in Kaibu.


ImJoy provides its own file manager which stores files in the browser, the file mamager is called "elFinder" (https://jupyter.imjoy.io/elFinder). To allow reading files locally, you need to:
 * First "upload" your local file to elFinder (Note: Your data won't go to any remote servers but stay inside the browser locally)
 * Then read the file in Python via a dedicated function called `open_elfinder()` (equvalent to `open` in Python).

See the example below.

In [32]:
# We need to first install imageio module
import micropip
await micropip.install(["imageio"])

In [33]:
from imjoy import api
import imageio
from imjoy_rpc.utils import open_elfinder

class ImJoyPlugin():
    async def setup(self):
        pass

    async def run(self, ctx):
        viewer = await api.createWindow(src="https://kaibu.org/#/app")

        async def form_submit_callback(fields):
            image_path = fields["image file"]
            image_file = open_elfinder(image_path, "rb")
            img = imageio.imread(image_file) 
            await viewer.view_image(img)
        
        async def select_file_callback():
            # await api.alert("select button clicked")
            # return True
            fm = await api.showDialog(src="https://jupyter.imjoy.io/elFinder/", name="Select a File")
            selections = await fm.getSelections()
            return selections[0]['path']
            
        await viewer.add_widget({
            "_rintf": True,
            "name": "My Form",
            "type": "form",
            "form_submit_callback": form_submit_callback,
            "fields": [
                {
                    "html": "<p><strong class='has-text-info'>Image Visualization Demo</strong><br>Please select an image file (or download one from <a href='https://images.proteinatlas.org/61448/1319_C10_2_blue_red_green.jpg' target='_blank'>here</a>)</p>"
                },
                {
                    "type": "button",
                    "label": "image file",
                    "callback": select_file_callback
                },
                # {
                #     "type": "number",
                #     "label": "cell size",
                #     "value": 10,
                #     "isRequired": False
                # },
            ],
        })
api.export(ImJoyPlugin())

<IPython.core.display.Javascript object>

<_GatheringFuture pending>

## Create cell annotations

Kaibu allows also annotate images with a set of different tools, for example, you can use it to create polygon annotations for cells, e.g. for training a deep learning model for cell segmentation.

In the following example, we use the function `viewer.add_shapes` to create an vector annotation layer in Kaibu, and then use `annotation_layer.get_features()` to obtain the polygons (in GeoJSON format) annotated by the user.

By using a utility module named `kaibu-utils` to convert GeoJSON annotations into mask images (a.k.a. label image, which is a common format for image segmentation).

In [19]:
import micropip
await micropip.install(["kaibu-utils"])

In [26]:
from imjoy import api
import imageio
from imjoy_rpc.utils import open_elfinder
from kaibu_utils import features_to_mask

image_masks = []

class ImJoyPlugin():
    async def setup(self):
        pass

    async def run(self, ctx):
        viewer = await api.createWindow(src="https://kaibu.org/#/app")

        async def form_submit_callback(fields):
            # image_path = fields["image file"]
            await api.alert("Done!")
        
        async def select_file_callback():
            fm = await api.showDialog(src="https://jupyter.imjoy.io/elFinder/", name="Select a File")
            selections = await fm.getSelections()
            image_path = selections[0]['path']
            image_file = open_elfinder(image_path, "rb")
            self.img = imageio.imread(image_file) 
            await viewer.view_image(self.img, name="Image")
            self.annotation_layer = await viewer.add_shapes([], name="Annotation")
            return image_path
        
        async def crate_mask_callback():
            features = await self.annotation_layer.get_features()
            mask = features_to_mask(features, self.img.shape[:2])
            await viewer.view_image(mask, name="Mask")
            image_masks.append({"mask": mask, "image": self.img})

        await viewer.add_widget({
            "_rintf": True,
            "name": "My Form",
            "type": "form",
            "form_submit_callback": form_submit_callback,
            "fields": [
                {
                    "html": "<p><strong class='has-text-info'>Image Visualization Demo</strong><br>Please select an image file</p>"
                },
                {
                    "type": "button",
                    "label": "image file",
                    "callback": select_file_callback
                },
                {
                    "type": "button",
                    "label": "create mask",
                    "callback": crate_mask_callback,
                    "isRequired": False
                },
                # {
                #     "type": "number",
                #     "label": "cell size",
                #     "value": 10,
                #     "isRequired": False
                # },
            ],
        })
api.export(ImJoyPlugin())

<IPython.core.display.Javascript object>

<_GatheringFuture pending>

In [29]:
image_masks

[{'mask': array([[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]], dtype=uint16),
  'image': Array([[[ 0,  0,  0],
          [ 0,  0,  0],
          [ 0,  0,  0],
          ...,
          [ 2,  3,  0],
          [ 2,  3,  0],
          [ 2,  3,  0]],
  
         [[ 0,  0,  0],
          [ 0,  0,  0],
          [ 0,  0,  0],
          ...,
          [ 2,  3,  0],
          [ 2,  3,  0],
          [ 2,  3,  0]],
  
         [[ 0,  0,  0],
          [ 0,  0,  0],
          [ 0,  0,  0],
          ...,
          [ 2,  3,  0],
          [ 2,  3,  0],
          [ 2,  3,  0]],
  
         ...,
  
         [[ 0,  0,  0],
          [ 0,  0,  0],
          [ 0,  0,  0],
          ...,
          [ 6, 13,  0],
          [ 0,  7,  0],
          [ 0,  5,  0]],
  
         [[ 0,  0,  0],
          [ 0,  0,  0],
          [ 0,  0,  0],
        